In [1]:
import json
import hashlib
from datetime import datetime
import urllib
import pandas as pd
import numpy as np
import math
import datetime as dt
import yfinance as yf

In [2]:
import logging

# Create a logger object
log = logging.getLogger()
log.setLevel(logging.DEBUG)  # Set the root logger level to DEBUG


# Create a console handler for printing logs to console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Set console handler level to INFO

# Define format for both handlers
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] - %(message)s')
console_handler.setFormatter(formatter)

# Add the handlers to the logger

log.addHandler(console_handler)



In [48]:
import yfinance as yf

# Ticker symbol for ITC
ticker_symbol = "^NSEBANK"  # For the National Stock Exchange of India

# Fetch historical data
data = yf.download(ticker_symbol, period="1d", interval="1m")
data['Change']=data['Close']-data['Open']
data['Pre_Change']= (data['Change']/data['Open'])*100


In [49]:
old_price_long = 0
old_price_short = 0
target_prencantage = 0.1
stop_loss_precentage = 0.2 
stop_loss_long = False
stop_loss_short = False
stop_loss_price_long = 0
stop_loss_price_short = 0
long_hold = False
short_hold = False
trade_book = []
qty = 30
brokerage = 20
stt = 0.025/100
transaction_charge = 0.000325/100
stamp = 0.003/100
sebi_charges = 10

for index, row in data.iterrows():

    
    current_price = row['Close']
    print(index,current_price)
    pre_incrase = ((current_price-old_price_long)/old_price_long)*100 if old_price_long != 0 else 0
    
    if (pre_incrase >= target_prencantage) and long_hold :
        long_hold = False
        profit_long = current_price-old_price_long
        print(f'selling at a price {current_price}. profit = {(current_price-old_price_long)*qty} \n')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})

    if pre_incrase<=-stop_loss_precentage and long_hold:
        long_hold = False
        stop_loss_long = True
        loss = old_price_long-current_price
        print('Stop Loss Triggered on long')
        print(f'Booked a Loss of {-loss} at price {current_price}')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':-loss*qty,'charges':charges,'profit_after_charges':(-loss*qty)-charges})

    if (not long_hold) and (not stop_loss_long):
        print('Got False going long')
        long_hold = True
        print(f'Buying at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_long = row['Close']
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stamp_rs = order_value*stamp
        charges = brokerage+stamp_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'long','price':current_price,'order_value':int(order_value),'profit':0,'charges':charges,'profit_after_charges':-charges})


    pre_dec = ((-current_price+old_price_short)/old_price_short)*100 if old_price_short != 0 else 0
    if (pre_dec >= target_prencantage) and short_hold:
        short_hold = False
        profit_short = -current_price+old_price_short
        print(f'selling at a price {current_price}. profit = {(-current_price+old_price_short)*qty} \n')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stamp_rs = order_value*stamp
        charges = brokerage+stamp_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_short*qty)-charges})
    
    if pre_dec<=-stop_loss_precentage and short_hold:
        short_hold = False
        stop_loss_short = True
        loss = +old_price_short-current_price
        print('Stop Loss Triggered on short')
        print(f'Booked a Loss of {-loss} at price {current_price}')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':-loss*qty,'charges':charges,'profit_after_charges':(-loss*qty)-charges})

    
    if (not short_hold) and (not stop_loss_short):
        
        print('Got False going short')
        short_hold = True
        print(f'Buying at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = row['Close']
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'short','price':current_price,'order_value':int(order_value),'profit':0,'charges':charges,'profit_after_charges':0-charges})
    
    

    

    if index.hour==15 and index.minute==29:
        if long_hold:
            long_hold = False
            profit_long = current_price-old_price_long
            print(f'Ending Trade for day {current_price}. profit = {(current_price-old_price_long)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stt_rs = order_value*stt
            charges = brokerage+stt_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})


        if short_hold:
            short_hold = False
            profit_short = -current_price+old_price_short
            print(f'Ending Short at a price {current_price}. profit = {(-current_price+old_price_short)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stamp_rs = order_value*stamp
            charges = brokerage+stamp_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_short*qty)-charges})
            
    
        break




    
    


2024-03-22 09:15:00+05:30 46725.75
Got False going long
Buying at a price 46725.75. total buy price 1401772.5 
 
Got False going short
Buying at a price 46725.75. total buy price 1401772.5 
 
2024-03-22 09:16:00+05:30 46704.19921875
2024-03-22 09:17:00+05:30 46686.8984375
2024-03-22 09:18:00+05:30 46700.94921875
2024-03-22 09:19:00+05:30 46695.0
2024-03-22 09:20:00+05:30 46719.6015625
2024-03-22 09:21:00+05:30 46735.44921875
2024-03-22 09:22:00+05:30 46720.69921875
2024-03-22 09:23:00+05:30 46720.75
2024-03-22 09:25:00+05:30 46663.8984375
selling at a price 46663.8984375. profit = 1855.546875 

Got False going short
Buying at a price 46663.8984375. total buy price 1399916.953125 
 
2024-03-22 09:26:00+05:30 46634.75
2024-03-22 09:27:00+05:30 46620.0
Stop Loss Triggered on long
Booked a Loss of -105.75 at price 46620.0
2024-03-22 09:28:00+05:30 46585.80078125
selling at a price 46585.80078125. profit = 2342.9296875 

Got False going short
Buying at a price 46585.80078125. total buy pric

In [50]:
old_price_long = 0
old_price_short = 0
target_prencentage = 0.1
stop_loss_precentage = 0.3 
stop_loss_long = False
stop_loss_short = False
stop_loss_price_long = 0
stop_loss_price_short = 0
long_hold = False
short_hold = False
trade_book = []
qty = 30
brokerage = 20
stt = 0.025/100
transaction_charge = 0.000325/100
stamp = 0.003/100
sebi_charges = 10

for index, row in data.iterrows():

    log.debug(f"variables values when running on_ticks getting ticks :- old_price_long : {old_price_long},old_price_short : {old_price_short},target_prencantage : {target_prencentage},stop_loss_precentage : {stop_loss_precentage},long_hold : {long_hold},short_hold : {short_hold},trade_book : {trade_book},qty : {qty}")
    #records_ticks(ticks['datetime'],ticks['open'],ticks['high'],ticks['low'],ticks['close'])

    

    current_price = row['Close']
    date = index
    

    per_change_long = ((current_price-old_price_long)/old_price_long)*100 if old_price_long != 0 else 0
    per_change_short = ((current_price-old_price_short)/old_price_short)*100 if old_price_short != 0 else 0
    price_change_long = current_price-old_price_long
    price_change_short = current_price-old_price_short
    log.info(f"% Change in Price for Long position = {per_change_long}")
    log.info(f"% Change in Price for Short position = {per_change_short} NOTE :: Negetive means profit in short trade same formula has been used to both long and short") 
    
    if (per_change_long >= target_prencentage) and long_hold :

        log.info(f"Exiting Long Trade since Both the Condition Satisfied per_change_long = {per_change_long} is greater than target_precentage = {target_prencentage} and Holding Position for Long is = {long_hold}")
        long_hold = False
        log.debug(f"setting long_hold to False.value after setting false = {long_hold}")
        profit_long = price_change_long
        log.info(f"Sold at {current_price} * {qty} = {current_price*qty}. Booked Profit of {profit_long} * {qty} = {profit_long*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long','price':{current_price},'order_value':{int(order_value)},'profit':{profit_long*qty})")

    if per_change_long <= -stop_loss_precentage and long_hold:

        log.warning(f"Stoploss on Long Position Triggered since Both Condition per_change_long = {per_change_long} is greater than Stoploss = {stop_loss_precentage} in absolute values and Long_hold is True = {long_hold}. NOTE per_change_long should be negetive since it is decreasing.")
        long_hold = False
        log.debug(f"Assigned False value to long_hold : {long_hold}")
        stop_loss_long = True
        log.debug(f"Changed stop_loss_long value to True since it is triggered. new value = {stop_loss_long}")
        loss = current_price-old_price_long
        log.warning(f"Exiting Long Position Booked a LOSS of {loss} * {qty} = {loss*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long STOPLOSS','price':current_price,'order_value':int(order_value),'profit':loss*qty})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long STOPLOSS','price':{current_price},'order_value':{int(order_value)},'profit':{loss*qty})")

    if (not long_hold) and (not stop_loss_long):

        log.info(f"Going Long Since not Holding and neither stoploss is triggered values :: long_hold : {long_hold}, stop_loss_long : {stop_loss_long}")
        long_hold = True

        log.info(f'Buying at a price {current_price} * {qty}. total buy price {qty*current_price} \n ')
        old_price_long = current_price
        log.debug(f"Setting global old_price_long to {current_price} and long_hold to {long_hold}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'long','price':current_price,'order_value':int(order_value),'profit':0})


    
    if (per_change_short <= -target_prencentage) and short_hold:
        log.debug(f"Exiting Short Trade since Both Condition Satisfied. per_change_short : {per_change_short} is greater than target_prencentage : {target_prencentage} and short hold is True Value : {short_hold}")
        short_hold = False
        log.debug(f'Set short_hold to False new Value = {short_hold}')
        profit_short = old_price_short-current_price 
        log.info(f"Exiting Trade at price {current_price} * qty {qty} = {current_price*qty}")
        log.info(f"Booked a profit of {profit_long} * {qty} = {profit_long*qty}")
        order_value = qty*current_price
        
        trade_book.append({'date':date,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty})



    if per_change_short >= stop_loss_precentage and short_hold:
        log.warning(f"Stoploss on Short Position Triggered since per_change_short : {per_change_short} is greater than stop_loss_precentage : {stop_loss_precentage}. in absolute term. and short_hold is True : {short_hold} NOTE :: per_change_should be positive since it is a short trade")
        short_hold = False
        log.debug('Assigned new value to short_hold False : {False}')
        stop_loss_short = True
        log.debug(f'Since Stoploss triggered setting stop_loss_short to Truew new value : {stop_loss_short}')
        loss_on_short = old_price_short-current_price
        log.warning(f'Exiting short Trade at price : {current_price}. Booked a loss of {loss_on_short} *qty = {loss_on_short*qty}')
        
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit short Stoploss','price':current_price,'order_value':int(order_value),'profit':loss*qty})

    
    if (not short_hold) and (not stop_loss_short):
        
        log.info(f"Going Short since Both short_hold : {short_hold} and stop_loss_short : {stop_loss_short} are False. ")
        short_hold = True
        log.info(f'Shorting at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = current_price
        log.debug(f"Setting old_short_price to {current_price}. value after setting {old_price_short}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'short','price':current_price,'order_value':int(order_value),'profit':0})

[2024-03-22 13:10:40,210][INFO] - % Change in Price for Long position = 0
[2024-03-22 13:10:40,211][INFO] - % Change in Price for Short position = 0 NOTE :: Negetive means profit in short trade same formula has been used to both long and short
[2024-03-22 13:10:40,211][INFO] - Going Long Since not Holding and neither stoploss is triggered values :: long_hold : False, stop_loss_long : False
[2024-03-22 13:10:40,212][INFO] - Buying at a price 46725.75 * 30. total buy price 1401772.5 
 
[2024-03-22 13:10:40,212][INFO] - Going Short since Both short_hold : False and stop_loss_short : False are False. 
[2024-03-22 13:10:40,213][INFO] - Shorting at a price 46725.75. total buy price 1401772.5 
 
[2024-03-22 13:10:40,214][INFO] - % Change in Price for Long position = -0.046121851976693794
[2024-03-22 13:10:40,214][INFO] - % Change in Price for Short position = -0.046121851976693794 NOTE :: Negetive means profit in short trade same formula has been used to both long and short
[2024-03-22 13:10:

In [52]:
trade_book_df = pd.DataFrame(trade_book)
trade_book_df

,date,type,price,order_value,profit
0,2024-03-22 09:15:00+05:30,long,46725.750000,1401772,0.000000
1,2024-03-22 09:15:00+05:30,short,46725.750000,1401772,0.000000
2,2024-03-22 09:25:00+05:30,exit short,46663.898438,1399916,1855.546875
3,2024-03-22 09:25:00+05:30,short,46663.898438,1399916,0.000000
4,2024-03-22 09:28:00+05:30,exit short,46585.800781,1397574,2342.929688
5,2024-03-22 09:28:00+05:30,short,46585.800781,1397574,0.000000
6,2024-03-22 09:36:00+05:30,exit long,46795.800781,1403874,2101.523438
7,2024-03-22 09:36:00+05:30,long,46795.800781,1403874,0.000000
8,2024-03-22 09:36:00+05:30,exit short Stoploss,46795.800781,1403874,-6300.000000
9,2024-03-22 10:12:00+05:30,exit long,46845.199219,1405355,1481.953125


In [53]:
data

,Open,High,Low,Close,Adj Close,Volume,Change,Pre_Change
Datetime,,,,,,,,
2024-03-22 09:15:00+05:30,46634.898438,46741.750000,46634.898438,46725.750000,46725.750000,0,90.851562,0.194815
2024-03-22 09:16:00+05:30,46724.550781,46736.898438,46700.250000,46704.199219,46704.199219,0,-20.351562,-0.043556
2024-03-22 09:17:00+05:30,46704.000000,46714.800781,46685.050781,46686.898438,46686.898438,0,-17.101562,-0.036617
2024-03-22 09:18:00+05:30,46689.550781,46701.601562,46681.351562,46700.949219,46700.949219,0,11.398438,0.024413
2024-03-22 09:19:00+05:30,46702.101562,46709.800781,46682.648438,46695.000000,46695.000000,0,-7.101562,-0.015206
...,...,...,...,...,...,...,...,...
2024-03-22 13:06:00+05:30,46899.750000,46902.000000,46879.949219,46886.500000,46886.500000,0,-13.250000,-0.028252
2024-03-22 13:07:00+05:30,46883.101562,46891.101562,46875.250000,46882.449219,46882.449219,0,-0.652344,-0.001391
2024-03-22 13:08:00+05:30,46879.101562,46889.148438,46873.648438,46878.199219,46878.199219,0,-0.902344,-0.001925


In [54]:
sum(trade_book_df['profit'])

3390.0

In [23]:
sum(trade_book_df['profit_after_charges'])

KeyError: 'profit_after_charges'